## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [0]:
!pip install haversine

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [0]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [0]:
bucks = pd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv")

Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [0]:
# answer goes here
hawaii = bucks[bucks['State/Province'] == 'HI']

The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
-121.64, 39.14  
-116.40, 34.13  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
-121.64, 39.14, (-121.64, 39.14)  
-116.40, 34.13, (-116.40, 34.13)  
...

In [0]:
# answer goes here
hawaii['Coordinates'] = list(zip(hawaii.Latitude, hawaii.Longitude))

hawaii.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)"
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)"
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(21.38, -157.94)"
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)"
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)"


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [0]:
# answer goes here
matrix = geo_sim_matrix(hawaii, col_name = 'Coordinates')
print(matrix)


index       17202       17203       17204  ...       17298       17299       17300
index                                      ...                                    
17202    0.000000    0.690934    0.643386  ...    5.329021    5.675488    6.470637
17203    0.690934    0.000000    0.944121  ...    5.548625    6.120771    6.580589
17204    0.643386    0.944121    0.000000  ...    4.710641    5.179933    5.831356
17205    0.944091    1.524305    0.690934  ...    4.555939    4.747018    5.790081
17206    1.460500    1.888212    0.944091  ...    3.921273    4.240594    5.146738
...           ...         ...         ...  ...         ...         ...         ...
17296  107.264542  106.962166  107.842613  ...  112.498970  112.833847  113.367599
17297   98.783943   98.552231   99.389603  ...  104.095673  104.204285  105.083298
17298    5.329021    5.548625    4.710641  ...    0.000000    2.170313    1.460423
17299    5.675488    6.120771    5.179933  ...    2.170313    0.000000    3.370790
1730

pandas.core.frame.DataFrame

For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [0]:
# answer goes here
Neighbor = []
Distance = []
for column in matrix:
  matrix1 = matrix[matrix[column] != 0]
  matrix2 = matrix1[column].sort_values()
  matrix3 = matrix2.iloc[:1]
  
  mindist = matrix3.values[0]
  neighbor = matrix3.index[0]
  Distance.append(mindist)
  Neighbor.append(neighbor)

[0.64338624, 0.6909342, 0.64338624, 0.64334226, 0.64334226, 3.9228303, 3.9228303, 0.65049416, 0.65049416, 1.5273017, 0.65049416, 0.64382493, 0.6909342, 0.6437811, 0.6909342, 0.64356196, 0.64382493, 0.94441974, 0.6909342, 0.6437811, 0.9443303, 0.6909342, 0.94441974, 0.64356196, 0.64382493, 0.6909342, 0.6909342, 0.6437811, 0.64382493, 2.1704302, 0.6437811, 0.94441974, 0.6909342, 0.6909342, 0.64382493, 0.6909342, 0.6437811, 2.5752997, 0.6909342, 0.6909342, 0.6909342, 0.64356196, 0.6909342, 1.9314747, 0.6909342, 0.6437811, 2.5752997, 0.6437811, 0.6909342, 0.64382493, 0.6909342, 0.9443303, 0.64382493, 1.2910336, 0.6909342, 0.6455168, 1.2910336, 0.94560313, 1.5252239, 0.6455168, 3.9212735, 3.9212735, 0.65069723, 0.65069723, 0.6909342, 15.612447, 1.3818684, 0.6432103, 0.6432103, 5.8519654, 0.94424057, 0.94424057, 0.94424057, 1.3818684, 1.3818684, 1.3818684, 8.059888, 0.65069723, 0.6909342, 0.6909342, 0.6909342, 2.4419143, 2.0426142, 2.0426142, 2.0426142, 0.94388086, 0.94388086, 1.4598792, 1.4

In [0]:
# Adams solution:
# np.fill_diagonal(matrix.values, np.nan)
# hawaii['Nearest Neighbor'] = matrix.idxmin()
# hawaii['Distance (Miles)'] = matrix.min(axis=1)

In [0]:
hawaii['nearest_neighbor'] = Neighbor
hawaii['distance_to_neighbor'] = Distance

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
hawaii.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,nearest_neighbor,distance_to_neighbor
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)",17204,0.643386
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)",17202,0.690934
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(21.38, -157.94)",17202,0.643386
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)",17206,0.643342
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)",17205,0.643342


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [0]:
# answer goes here


hawaii.sort_values(by='distance_to_neighbor', ascending= False)


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,nearest_neighbor,distance_to_neighbor
17295,Starbucks,21070-104953,Queens Marketplace,Company Owned,"69-201 Waikoloa Beach Drive, #1001 K-1",Waikoloa,HI,US,967385810,808-886-1888,GMT-10:00 Pacific/Honolulu,-155.88,19.91,"(19.91, -155.88)",17267,15.612447
17267,Starbucks,21030-58899,Parker Ranch Center,Company Owned,67-1185 Mamalahoa Highway D108,Kamuela,HI,US,967438427,808-887-6409,GMT-10:00 Pacific/Honolulu,-155.67,20.02,"(20.02, -155.67)",17295,15.612447
17278,Starbucks,21071-107523,Poipu Shopping Village,Company Owned,"2360 Kiahuna Plantation Drive, Suites E-70 & E-80",Koloa,HI,US,967569552,(808) 742-5144,GMT-10:00 Pacific/Honolulu,-159.46,21.88,"(21.88, -159.46)",17284,8.059888
17291,Starbucks,21040-85520,Pukalani Terrace Center,Company Owned,55 Pukalani Street,Pukalani,HI,US,96768,808-573-5458,GMT-10:00 Pacific/Honolulu,-156.34,20.84,"(20.84, -156.34)",17261,7.322419
17294,Starbucks,21073-102259,Waianae Mall - Farrington Hwy,Company Owned,"86-120 Farrington Highway, Waikoloa Beach Resort",Waianae,HI,US,967923000,(808) 697-8279,GMT-10:00 Pacific/Honolulu,-158.18,21.44,"(21.44, -158.18)",17292,6.739833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)",17204,0.643386
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)",17205,0.643342
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)",17206,0.643342
17269,Starbucks,20315-202708,Windward Mall,Company Owned,"46-056 Kamehameha, C01",Kaneohe,HI,US,96744,(808) 235-2618,GMT-10:00 Pacific/Honolulu,-157.81,21.42,"(21.42, -157.81)",17270,0.643210
